In [3]:
from sklearn.metrics import roc_curve,auc,confusion_matrix, precision_recall_fscore_support
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import precision_score, roc_auc_score, recall_score
from sklearn.metrics import accuracy_score
from imblearn.metrics import specificity_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler

In [2]:
#prediction
def pred_rf_test(rf, cov_train_final, cov_test_final, thresh = 0.5, race = 'patient_race_all'):
    
    p_train = rf.predict_proba(cov_train_final.loc[:,~cov_train_final.columns.isin(["ssi"])])[:,1]
    p_test = rf.predict_proba(cov_test_final.loc[:, ~cov_test_final.columns.isin(["ssi"])])[:,1]
    y_pred = rf.predict(cov_test_final.loc[:, ~cov_test_final.columns.isin(["ssi"])])

    fpr_train, tpr_train, _ = roc_curve(cov_train_final["ssi"], p_train)
    roc_train = auc(fpr_train, tpr_train)

    fpr_test, tpr_test, _ = roc_curve(cov_test_final["ssi"], p_test)
    roc_test = auc(fpr_test, tpr_test)
    
    plt.plot(fpr_test, tpr_test, label = str(race[13:]))
    plt.xlabel("False-positive rate")
    plt.ylabel("True-positive rate")
    plt.gca().set_title("ROC-AUC:" + "".join(str(roc_test)[:5]))
    plt.legend()

    pr, re, thresh_test = precision_recall_curve(cov_test_final["ssi"], p_test)
    auprc = auc(re, pr)
    
    plt.figure()
    plt.plot(re, pr, label = str(race[13:]))
    plt.xlabel("Recall")
    plt.ylabel("Precision")
    plt.gca().set_title("auprc:" + "".join(str(auprc)[:5]))
    plt.legend()

    #prec = precision_score(cov_test_final["ssi"], y_pred, average = 'binary')
    #rec = recall_score(cov_test_final["ssi"], y_pred, average = 'binary')
    #spec = specificity_score(cov_test_final["ssi"], y_pred, average = 'binary')
    
    #rep = classification_report(cov_test_final["ssi"], y_pred)
    '''spec_arr = []
    for t in thresh_test:
        temp_pred = []
        for y in p_test:
            if y >= t:
                temp_pred.append(1)
            else:
                temp_pred.append(0)
        conf_matrix = confusion_matrix(cov_test_final["ssi"], temp_pred)
        TP = conf_matrix[1][1]
        TN = conf_matrix[0][0]
        FP = conf_matrix[0][1]
        FN = conf_matrix[1][0]
        conf_specificity = (TN / float(TN + FP))
        spec_arr.append(conf_specificity)

    spec_test = sum(spec_arr)/len(spec_arr)'''

    temp_pred = []
    for y in p_test:
        if y >= thresh:
            temp_pred.append(1)
        else:
            temp_pred.append(0)
    conf_matrix = confusion_matrix(cov_test_final["ssi"], temp_pred)
    TP = conf_matrix[1][1]
    TN = conf_matrix[0][0]
    FP = conf_matrix[0][1]
    FN = conf_matrix[1][0]
    spec = (TN / float(TN + FP))
    prec = (TP / float(TP + FP))
    rec = (TP / float(TP + FN))
                
    
    '''print("AUPRC : ", auprc)
    plt.figure()
    plt.plot(re, pr)
    plt.xlabel("Recall")
    plt.ylabel("Precision")
    plt.title(sensitive_var)
    plt.show()'''
    
    return (roc_train, roc_test, prec, rec, spec, auprc, pr, re, thresh_test\
            , p_test)

In [ ]:
def pred_lr_test(lr, cov_train_final, cov_test_final, thresh = 0.5, race = 'patient_race_all'):
    
    scaler = StandardScaler()
    X = cov_train_final.drop(['ssi'], axis = 1)
    scaler.fit(X)
    X = scaler.transform(X)
    
    X_test = cov_test_final.drop(['ssi'], axis = 1)
    X_test = scaler.transform(X_test)
    
    p_train = lr.predict_proba(X)[:,1]
    p_test = lr.predict_proba(X_test)[:,1]
    y_pred = lr.predict(X_test)

    fpr_train, tpr_train, _ = roc_curve(cov_train_final["ssi"], p_train)
    roc_train = auc(fpr_train, tpr_train)

    fpr_test, tpr_test, auc_thresh = roc_curve(cov_test_final["ssi"], p_test)
    roc_test = auc(fpr_test, tpr_test)
    
    plt.plot(fpr_test, tpr_test, label = str(race[13:]))
    plt.xlabel("False-positive rate")
    plt.ylabel("True-positive rate")
    plt.gca().set_title("ROC-AUC:" + "".join(str(roc_test)[:5]))
    plt.legend()

    pr, re, thresh_test = precision_recall_curve(cov_test_final["ssi"], p_test)
    auprc = auc(re, pr)
    
    plt.figure()
    plt.plot(re, pr, label = str(race[13:]))
    plt.xlabel("Recall")
    plt.ylabel("Precision")
    plt.gca().set_title("auprc:" + "".join(str(auprc)[:5]))
    plt.legend()

    #prec = precision_score(cov_test_final["ssi"], y_pred, average = 'binary')
    #rec = recall_score(cov_test_final["ssi"], y_pred, average = 'binary')
    #spec = specificity_score(cov_test_final["ssi"], y_pred, average = 'binary')
    
    temp_pred = []
    for y in p_test:
        if y >= thresh:
            temp_pred.append(1)
        else:
            temp_pred.append(0)
    conf_matrix = confusion_matrix(cov_test_final["ssi"], temp_pred)
    TP = conf_matrix[1][1]
    TN = conf_matrix[0][0]
    FP = conf_matrix[0][1]
    FN = conf_matrix[1][0]
    spec = (TN / float(TN + FP))
    prec = (TP / float(TP + FP))
    rec = (TP / float(TP + FN))
                  
    
    return (roc_train, roc_test, prec, rec, spec, auprc, pr, re, thresh_test\
            , p_test, fpr_test, tpr_test, auc_thresh)

In [ ]:
def pred_sgd_test(lr, cov_train_final, cov_test_final, thresh = 0.5, race = 'patient_race_all'):
    
    scaler = StandardScaler()
    X_test = cov_test_final.drop(['ssi'], axis = 1)
    X_test = scaler.fit_transform(X_test)
    
    X = cov_train_final.drop(['ssi'], axis = 1)
    X = scaler.fit_transform(X)
    
    p_train = lr.predict_proba(X)[:,1]
    p_test = lr.predict_proba(X_test)[:,1]
    y_pred = lr.predict(X_test)

    fpr_train, tpr_train, _ = roc_curve(cov_train_final["ssi"], p_train)
    roc_train = auc(fpr_train, tpr_train)

    fpr_test, tpr_test, _ = roc_curve(cov_test_final["ssi"], p_test)
    roc_test = auc(fpr_test, tpr_test)
    
    plt.plot(fpr_test, tpr_test, label = str(race[13:]))
    plt.xlabel("False-positive rate")
    plt.ylabel("True-positive rate")
    plt.gca().set_title("ROC-AUC:" + "".join(str(roc_test)[:5]))
    plt.legend()

    pr, re, thresh_test = precision_recall_curve(cov_test_final["ssi"], p_test)
    auprc = auc(re, pr)

    #prec = precision_score(cov_test_final["ssi"], y_pred, average = 'binary')
    #rec = recall_score(cov_test_final["ssi"], y_pred, average = 'binary')
    #spec = specificity_score(cov_test_final["ssi"], y_pred, average = 'binary')
    
    temp_pred = []
    for y in p_test:
        if y >= thresh:
            temp_pred.append(1)
        else:
            temp_pred.append(0)
    conf_matrix = confusion_matrix(cov_test_final["ssi"], temp_pred)
    TP = conf_matrix[1][1]
    TN = conf_matrix[0][0]
    FP = conf_matrix[0][1]
    FN = conf_matrix[1][0]
    spec = (TN / float(TN + FP))
    prec = (TP / float(TP + FP))
    rec = (TP / float(TP + FN))
                  
    
    return (roc_train, roc_test, prec, rec, spec, auprc, pr, re, thresh_test\
            , p_test)

In [ ]:
def pred_xg_test(xg, cov_train_final, cov_test_final, thresh = 0.5, race = 'patient_race_all'):
    
    X = cov_train_final.loc[:,~cov_train_final.columns.isin(["ssi"])]
    Y = cov_train_final["ssi"]
    X_test = cov_test_final.loc[:,~cov_test_final.columns.isin(["ssi"])]
    
    p_train = xg.predict_proba(X)[:,1]
    p_test = xg.predict_proba(X_test)[:,1]
    y_pred = xg.predict(X_test)

    fpr_train, tpr_train, _ = roc_curve(cov_train_final["ssi"], p_train)
    roc_train = auc(fpr_train, tpr_train)

    fpr_test, tpr_test, _ = roc_curve(cov_test_final["ssi"], p_test)
    roc_test = auc(fpr_test, tpr_test)
    
    plt.plot(fpr_test, tpr_test, label = str(race[13:]))
    plt.xlabel("False-positive rate")
    plt.ylabel("True-positive rate")
    plt.gca().set_title("ROC-AUC:" + "".join(str(roc_test)[:5]))
    plt.legend()

    pr, re, thresh_test = precision_recall_curve(cov_test_final["ssi"], p_test)
    auprc = auc(re, pr)
    
    plt.figure()
    plt.plot(re, pr, label = str(race[13:]))
    plt.xlabel("Recall")
    plt.ylabel("Precision")
    plt.gca().set_title("auprc:" + "".join(str(auprc)[:5]))
    plt.legend()

    #prec = precision_score(cov_test_final["ssi"], y_pred, average = 'binary')
    #rec = recall_score(cov_test_final["ssi"], y_pred, average = 'binary')
    #spec = specificity_score(cov_test_final["ssi"], y_pred, average = 'binary')
    
    temp_pred = []
    for y in p_test:
        if y >= thresh:
            temp_pred.append(1)
        else:
            temp_pred.append(0)
    conf_matrix = confusion_matrix(cov_test_final["ssi"], temp_pred)
    TP = conf_matrix[1][1]
    TN = conf_matrix[0][0]
    FP = conf_matrix[0][1]
    FN = conf_matrix[1][0]
    spec = (TN / float(TN + FP))
    prec = (TP / float(TP + FP))
    rec = (TP / float(TP + FN))
                  
    
    return (roc_train, roc_test, prec, rec, spec, auprc, pr, re, thresh_test\
            , p_test)